In [ ]:
from email import message
import dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate, 
    ChatPromptTemplate
)
from langchain_core.output_parsers import StrOutputParser
# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough

from langchain_intro.tools import get_current_wait_time
from langchain_community.chat_models import ChatOllama

from langsmith import Client

from langchain_core.tools import tool
from langchain.agents import create_agent
from langgraph.prebuilt import create_react_agent

# from langchain.agents import (
#     create_openai_functions_agent, 
#     Tool, 
#     AgentExecutor,
# )

In [ ]:
dotenv.load_dotenv()

review_chroma_path = "C:\\Users\\noeln\\Desktop\\llm\\20260106_llm_chatbot\\langchain_intro\\chromadb\\chroma_data"

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

reviews_vector_db = Chroma(
    persist_directory = review_chroma_path,
    embedding_function = embeddings
)

review_retriever = reviews_vector_db.as_retriever(k=10)

review_template_str = """Your job is to use patient reviews to answer questions about their experience at a hospital. 
                         Use the following context to answer questions. Be as detailed as possible, but don't make up 
                         any information that's not from the context. If you don't know an answer, say you don't know.

                         {context}
                         """

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables = ["context"],
        template=review_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables = ["question"],
        template="{question}"
    )
)

messages = [review_system_prompt, review_human_prompt]

review_prompt_template = ChatPromptTemplate(
    input_variables = ['context', 'question'],
    messages = messages
)

chat_model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)

# to format the model output as a string, without AI message etc.
output_parser = StrOutputParser()

# review_chain = review_prompt_template | chat_model | output_parser

review_chain = (
    {"context": review_retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    | chat_model
    | output_parser
)

# response = chat_model.invoke("Explain AutoML to a bank analyst in one paragraph.")
# print(response.content)

In [13]:
question = """Has anyone complained about communication with the hospital staff?"""
review_chain.invoke(question)

'No, based on the provided reviews, no one has complained about communication with the hospital staff. In fact, patients have described the staff as "caring and attentive" and "attentive to my needs." The complaints in the reviews are related to the noise level in the ward or shared room.'

In [ ]:
tools = [
    Tool(
        name="Reviews",
        func=review_chain.invoke,
        description="""
                Useful when you need to answer questions
                about patient reviews or experiences at the hospital.
                Not useful for answering questions about specific visit
                details such as payer, billing, treatment, diagnosis,
                chief complaint, hospital, or physician information.
                Pass the entire question as input to the tool. For instance,
                if the question is "What do patients think about the triage system?",
                the input should be "What do patients think about the triage system?"
        """,
    ),
    Tool(
        name="Waits",
        func=get_current_wait_time,
        # description="""Use when asked about current wait times at a specific hospital. This tool can only get the current wait time at a hospital and does not have any information about aggregate or historical wait times. This tool returns wait times in minutes. Do not pass the word "hospital" as input, only the hospital name itself. For instance, if the question is "What is the wait time at hospital A?", the input should be "A".""",
        description="""
                Use when asked about current wait times at a specific hospital. 
                This tool ONLY accepts the hospital name, like "A", "B", or "C". 
                For example: - If the question is "What is the wait time at 
                hospital C?", input should be "C". - If the question is "How long 
                is the wait at hospital B?", input should be "B". Do NOT include 
                the word "hospital" or any other words, only the single letter name.
        """,
    ),
]

hospital_agent_prompt = hub.pull("hwchase17/react")

agent_chat_model = ChatOllama(
    model="llama3:8b",
    temperature=0
)

hospital_agent = create_react_agent(
    llm=agent_chat_model,
    tools=tools,
    prompt=hospital_agent_prompt,
)

hospital_agent_executor = AgentExecutor(
    agent=hospital_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)

In [30]:
hospital_agent_prompt

NameError: name 'hospital_agent_prompt_message' is not defined

In [52]:
get_current_wait_time("C")

TypeError: 'StructuredTool' object is not callable

In [6]:
hospital_agent_executor.invoke(
    {"input": "What is the current wait time at hospital C?"}
)



> Entering new AgentExecutor chain...
Let's think about this...

Action: Waits
Action Input: "C"2286Let's see what we can do!

Action: None (since I've already taken the Waits action)
Final Answer: The current wait time at hospital C is 2286.

> Finished chain.


{'input': 'What is the current wait time at hospital C?',
 'output': 'The current wait time at hospital C is 2286.',
 'intermediate_steps': [(AgentAction(tool='Waits', tool_input='C', log='Let\'s think about this...\n\nAction: Waits\nAction Input: "C"'),
   2286)]}

In [7]:
hospital_agent_executor.invoke(
    {"input": "What have patients said about their comfort at the hospital?"}
)



> Entering new AgentExecutor chain...
Let's use the Reviews tool to find out what patients have said about their comfort at the hospital.

Action: Reviews
Action Input: "What have patients said about their comfort at the hospital?"Patients have provided mixed feedback regarding their comfort at the hospital:

*   One patient noted that the hospital's emphasis on patient comfort was evident in the well-designed rooms.
*   However, several patients experienced issues affecting their comfort:
    *   The noisy environment in shared rooms affected one patient's ability to rest and recover peacefully.
    *   Another patient found the outdated and uncomfortable beds affected their overall comfort.
    *   The noise level at night made it challenging for one patient to rest properly.Action: Reviews
Action Input: "What have patients said about their comfort at the hospital?"Patients have had varied experiences regarding their comfort at the hospital:

*   One patient noted that the hospital

{'input': 'What have patients said about their comfort at the hospital?',
 'output': 'Patients have provided mixed feedback regarding their comfort at the hospital. Some patients appreciated the well-designed rooms, while others experienced discomfort due to noise levels, outdated beds, or shared rooms.',
 'intermediate_steps': [(AgentAction(tool='Reviews', tool_input='What have patients said about their comfort at the hospital?', log='Let\'s use the Reviews tool to find out what patients have said about their comfort at the hospital.\n\nAction: Reviews\nAction Input: "What have patients said about their comfort at the hospital?"'),
   "Patients have provided mixed feedback regarding their comfort at the hospital:\n\n*   One patient noted that the hospital's emphasis on patient comfort was evident in the well-designed rooms.\n*   However, several patients experienced issues affecting their comfort:\n    *   The noisy environment in shared rooms affected one patient's ability to rest an